In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
import pathlib
import glob

import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import jax
import jax.numpy as jnp
import equinox as eqx

In [ ]:
from mc2.utils.data_inspection import (
    get_available_material_names, get_file_overview, filter_file_overview,
    load_single_file, load_and_process_single_from_full_file_overview,
)
from mc2.utils.data_plotting import (
    plot_single_sequence
)
from mc2.data_management import FrequencySet, MaterialSet, DataSet

## TODOs:
- central place to store the measurment data -> ineffective for everyone to carry it around (Full size will be roughly 20 GB, could be done with LFS i guess?)
- How should the data be stored? What are our requirements for the data format?
    - how big is the actual dataset if stored as one single file? Can you properly work with this?
    - What filters are necessary?
    - What will generally be variables and what will be semi-constant?
- I guess there is no reason to not put T, B, and H information into a single file
- The only question would be how to deal with different temperature values? since they are semi constant

Current plans for data filters as of now:
- material
- frequency
- temperature

## File inspection

In [ ]:
frequencies = np.array([50, 80, 125, 200, 320, 500, 800]) * 1e3  # 50 - 800 kHz
sampling_frequency = 16 * 1e6  # 16 MHz

In [ ]:
raw_data_path = pathlib.Path("../../data/raw")

In [ ]:
material_names = get_available_material_names(raw_data_path)

full_file_overview = get_file_overview(
    raw_data_path=raw_data_path,
    material_names=material_names,
    frequencies=frequencies,
)
full_file_overview

## Data loading / processing:

In [ ]:
ov = filter_file_overview(
    full_file_overview,
    material_name="3C90",
    data_type=["B", "T", "H"],
    frequency=[80_000, 320_000],
)
ov

In [ ]:
dataset = data_set = DataSet.load_from_raw_data(
    file_overview=full_file_overview,
    material_names=material_names,
    frequencies=frequencies,
)

# three_mat_dataset = DataSet.load_from_raw_data(
#     file_overview=full_file_overview,
#     material_names=material_names[-3:],
#     frequencies=frequencies,
# )

In [ ]:
# store dataset 
are_you_sure = False

processed_data_path = pathlib.Path("../../data/processed") / "data.pickle"

if are_you_sure:
    dataset.save_to_file(processed_data_path)
    print("Data was saved at..", processed_data_path)
else:
    print("WARNING: Data was not saved!")